<a id="inicio"></a>
<img src="./figs/barra_uclm_esiiab.png" alt="Banner UCLM - ESIIAB" align="right">

<br><br><br>
<h1><font color="#B30033" size=5>Intelligent Systems - Course 2021-2022</font></h1>



<h1><font color="#B30033" size=5>Assignment 3: Policy learning</font></h1>


<br><br>
<div style="text-align: left">
<font color="#4E70BE" size=3>Lecturers:</font><br>
<ul>
  <li><font color="#4E70BE" size=3>Juan Carlos Alfaro Jiménez</font><br></li>
  <li><font color="#4E70BE" size=3>Guillermo Tomás Fernández Martín</font><br></li>
  <li><font color="#4E70BE" size=3>José Antonio Gámez Martín</font><br></li>
  <li><font color="#4E70BE" size=3>Ismael García Varea</font><br></li>
  <li><font color="#4E70BE" size=3>Luis González Naharro</font><br></li>
  <li><font color="#4E70BE" size=3>Jesús Martínez Gómez</font><br></li>    
</ul>
<font color="#4E70BE" size=3>Students:</font><br>
<ul>
  <li><font color="#4E70BE" size=3>Pablo Lario Gómez</font><br></li>
  <li><font color="#4E70BE" size=3>Diego Miguel López</font><br></li>
</ul>
</div>

<br>

## Introduction
In this assignment we will study and put into practice the algorithms studied in unit 5 (value/policy iteration and reinforcement lerning) which are able to learn a policy to solve a given problem. We will use a problem based on the same environment that the one used in Assignments 1 and 2. 

Analyzing and comparing the performance of the algorithms by running them over different instances of the problem is also a goal of this Assignment.

## Problem description
We use the same environment as in previous assignments but now, instead of pieces of garbage, a set of gems is scattered over the maze. Furthermore, the **value** of these gems is different, being even zero for some gems. In this updating of our problem, the episode ends when the agent enters in a cell ocuppied by a gem, regardless of its value.

Our goal is to learn a policy which maximizes the long-term reward obtained by the agent, which is computed as the value associated to the collected gem minus the cost (negative rewards) associated to all the visited non-final states. 

Thus, the maze is a grid of size N x M formed by a set of cells, some of which can be occupied by walls, which cannot be crossed. The rest of the cells will be empty and they will represent  free space. For now, the robot can only move horizontally or vertically. In addition, we can have cells with gems, being the goal of our robot to collect the one of highest value. 

The objective of our robot is to collect the most valuable gem as fast as possible, in particular when $\gamma$ is less than 1. Furthermore, the environment can be **noisy**, that is, the output of the proposed action may not be as expected. 

In order to implement our robot we have to take into account that: 
- The robot can start in a random cell of the map, but we will train our agent to solve a particular scenario.
- The robot can move horizontally or vertically in the maze.
- The robot cannot cross walls nor go beyond the bounds of the maze.
- As the goal is achieved once the robot collect the first gem, it should avoid the cells containing less valuable gems. 
- The rewards for final and non-final states are provided by the environment.
- On the contrary to previous assignments, our goal now is not to provide a path, but a policy for **all** the states in our environment (maze). 


## Provided code

In the following cells, we provide you some of the clases, implemented in `Python`, that will help you in the development of this assignment. 

First, we will import the necessary classes we need from the Python libraries

In [17]:
import math
import copy
import time

Next, we will import some custom functions from the `utils.py` file. You don't need to modify those functions for the code to work, but feel free to have a look at them if you are curious. This code is identical to the one provided in Assignments 1 and 2.

In [18]:
from utils import *

Finally, we will import some third party libraries. We will use those to display the problem in a graphical environment. In order to do that, we will use the magic functions from jupyter to install the library from inside the notebook.

In [19]:
!pip install ipythonblocks

You should consider upgrading via the 'C:\Users\49427234\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [20]:
from ipythonblocks import BlockGrid, colors
from IPython.display import clear_output

In order to complete the requested policy-learning algorithms, in the following cells we provide you some fundamental classes. 

#### Class `Action`
This class provides the **representation of the actions** that will be performed by the robot. You don't have to modify the code of this class. The possible actions will be: "UP", "DOWN", "RIGHT", "LEFT". This class is almost identical to the one provided in the previous assignments, but now there is no cost associated to any action (but to visiting states: rewards). To help us with policy visualization, we have added a new method that just returns a character which is a graphical representation of each action.

In [21]:
class Action:
    #actions = ["UP", "DOWN", "RIGHT", "LEFT"]

    def __init__(self, move):
        self.move = move

    def __str__(self):
        return f"({self.move})"
    
    def getChar(self):
        if self.move == "UP":
            return "↑"
        elif self.move == "DOWN":
            return "↓"
        elif self.move == "RIGHT":
            return "→"
        elif self.move == "LEFT":
            return "←"
    

#### Class `State`
This class provides the **representation of a state** in the search space. In this problem, a state is defined in a simpler way than in the previous assignments. Now, the state is simply defined by the position of the agent. For coherence, we will use also **states** to represent the position of the gems. You don't have to modify the code of this class.

Notice that the method `applyAction` is not needed anymore in this class. We will comment on that in the `Problem` class, but by now, try guessing the reason behind this 😉

In [22]:
class State:

    def __init__(self, pos):
        self.pos = pos

    # equals method. Returns true if the states are the same. 
    # Used for the hash table comparison, compares if both states are equal
    def __eq__(self, state):
        return self.pos == state.pos

    def __str__(self):
        return f"Position: {self.pos}"

    # hash method. Useful to index data structures that uses a hash function 
    #    to index elements, i.e. a Set()
    def __hash__(self):
        return int( math.pow(10,3)*self.pos[0] + self.pos[1])

    

## Implementation
In the following cells, we provide you some classes and pieces of code that you will have to complete as a part of this assignment.


#### Class `Problem`
This class provides the **representation of the learning problem**. As you may notice, in this assignment, this class is a bit more complex than in the previous ones. The reason is that now we have to deal with an **uncertain environment**. 

It contains the size of the maze (`rows` and `cols`), the `initialState`, the `maze`, the `gems` positions, the `gemsRewards` and also the `moveProb` value, which codifies how uncertain is the environment. Thus, `moveProb=1.0` means a deterministic environment, but e.g. `moveProb=0.8` means that the selected action will be actually applied 80% of the times, while 20% of the times another possible action will be applied. You will have to implement some parts of this class. 

The class `Problem` also has some interesting methods:

- `getPossibleActions(state)` will generate all the actions that can be applied in a given state. **You must code this method**. Notice that it is somehow similar to `getSucessors` in previous assignments. To make things easier we recommend programming this method to return a *pair of lists, where the first list will contain the possible actions, and the second list will contain the states reached with the actions from the first list*.

- `getTransitionDistribution(state, action)` will return the transition probability distribution when trying to apply action `action` in state `state`, as a *list of triplets, where each triplet consists of a possible action, the state that action leads to, and the probability to land on that state*. Take a look to the code to understand its meaning.

- `getReward(state)` returns *a number* representing the reward associated to the given state, regardless of whether it is a final state or not.

- `applyAction(state, action)` returns the state obtained when we give the order to execute the action `action` in state `state`. However, because of the (probably) noisy environment, the result can be an unexpected outcome. This is the reason this method is now in the `Problem` class, because it needs to make use of the transition probabilities.

- There are some other useful methods in this class, we encourage you to take a look at them.

In [23]:
class Problem:

    actions = ["UP", "DOWN", "RIGHT", "LEFT"]

    def __init__(self, rows, cols, seed, maxDivisions, gemsCount, transProbability, filename=""):

        if (filename != ""):
            self.rows, self.cols, self.maze = readProblemInstance(filename)
            print('Problem read with size',rows,'x',cols)
        else:
            self.rows = rows
            self.cols = cols
            self.maze = getProblemInstance(rows, cols, maxDivisions, gemsCount, seed)

        self.moveProb = transProbability # probability of actually applying the desired action
        self.gems = []
        self.gemsRewards = []

        for r in range(rows):
            for c in range(cols):
                if self.maze[r][c] == 2:
                    init_point = (r,c)
                elif self.maze[r][c] == 3:
                    self.gems.append(State((r,c)))
                    
        self.initialState = State(init_point)

        # setting rewards for final states.
        # about 50% of the gems have no value
        
        for gem in self.gems:
            rw = 5*manhattan_distance(self.initialState.pos, gem.pos)
            self.gemsRewards.append(int(rw*random.randint(0, 1)))
        
        
        
    # returns the possible actions from a given state and the states achived by them
    
    def getPossibleActions(self, state):

        possibleActions = []
        neighbours = []
        
        #
        # YOUR CODE HERE
        #
        
        auxPos = ()

        for a in self.getActions():
            # We first obtain the pos of the expected state after applying action a
            if (a == "UP"):
                auxPos = (state.pos[0]-1,state.pos[1])
            elif (a == "DOWN"):
                auxPos = (state.pos[0]+1,state.pos[1])
            elif (a == "RIGHT"):
                auxPos = (state.pos[0],state.pos[1]+1)
            elif (a == "LEFT"):
                auxPos = (state.pos[0],state.pos[1]-1)
            else:
                print("\n*** ERROR ***: Action " + a + "  is not allowed .....\n")
                sys.exit()
            
            # Now we check if that state is a valid one or not (a valid one will be one that is not a wall or is not outside the bounds of the maze)
            # We check if the state with pos=auxPos is inside the maze and not out of bounds
            if (auxPos[0] < self.rows and auxPos[0] >= 0 and auxPos[1] < self.cols and auxPos[1] >= 0):
              cell = self.maze[auxPos[0]][auxPos[1]]

              # We check if the new state will not lead to a wall
              if cell != 1:
                  possibleActions.append(Action(a))
                  neighbours.append(State(auxPos))
            
        # 
        # END OF YOUR CODE
        #
                
        return possibleActions, neighbours
    
    
    # returns the transition distribution from a given state when trying to apply an action
    # for each possible action a tuple (action, resulting state, transition probability) is returned
        
    def getTransitionDistribution(self, state, action):
        
        distribution = []
        posActions, neighbours = self.getPossibleActions(state)
        
        if len(posActions)>1:
            noise = 1.0 - self.moveProb
            for posAction, posNeighbour in zip(posActions,neighbours): 
                if posAction.move == action.move:
                    distribution.append((posAction, posNeighbour, self.moveProb))
                else:
                    distribution.append((posAction, posNeighbour, noise/float(len(posActions)-1)))
        else:
            distribution.append((posActions[0], neighbours[0], 1.0))            
        
        return distribution
    
    
    # apply the given action (under possibly a noisy environment) a returns the obtained state
    
    def applyAction(self, state, action):
        distribution = self.getTransitionDistribution(state, action)
        
        # we sample the actual action to be applied from the transition distribution probability
        
        distWeights = [probability for action, state, probability in distribution] 
        selectedDistribution = random.choices(distribution, weights=distWeights)[0]            
        selectedState = copy.deepcopy(selectedDistribution[1])    
        
        return selectedState
        
        
    # additional methods    
        
    def getInitialState(self):
        return self.initialState

    def getActions(self):
        return self.actions

    def isFinal(self, st):
        '''
        check if the given state is final or not
        '''
        return (st in self.gems)
    
    def getGems(self):
        return self.gems
    
    def getGemsRewards(self):
        return self.gemsRewards
    
    def getReward(self, st):
        try:
            p = self.gems.index(st)
        except:
            p = -1
            
        if p>=0:
            return self.gemsRewards[p]
        else:
            return -1
        

#### Class `ValueIteration`

This class will contain the ValueIteration method as explained in the lectures. You will have to code most part of this class. **Hint: follow as much as possible the pseudocode of the method provided in the slides.**

Remember that in this method we can exploit the knowledge about the environment (e.g. the states), the transition and reward models.

Notice that a `dictionary` data structure is used to store both the policy (entries are pairs <state,action>) and the utilities (entries are pairs <state,value>).


In [24]:
# Value Iteration

class ValueIteration:
    def __init__(self, problem, gamma, threshold):
        self.problem = problem
        self.gamma = gamma
        self.convergenceThreshold = threshold 
        self.policy = {}
        self.utilities = {}
        self.nfStates, self.fStates = self.getAllStates()
        self.policy, self.utilities = self.initPolicyAndUtilities()

        
    # check the maze and collect all positions non occupied by walls and not being final states (gems)    
    # Final states (gems) are returned on another list 
    def getAllStates(self):
        
        nfStates = []  # non final states
        fStates = []   # final states 
        
        for r in range(self.problem.rows):
            for c in range(self.problem.cols):
                if (self.problem.maze[r][c] != 1):
                    if (self.problem.maze[r][c] !=3):
                        nfStates.append(State((r,c)))
                    else: 
                        fStates.append(State((r,c)))
        
        return nfStates, fStates
        
        
    # return the utilities for all the states.
    def getUtilities(self):
        return self.utilities
    
    def initPolicyAndUtilities(self): 
        policy = {}
        utilities = {}
        
        # YOUR CODE HERE
        # Initialize the utilities and policy dictionaries.
        
        for s in self.nfStates:
            policy[s] = None
            utilities[s] = 0

        for s in self.fStates:
            policy[s] = None
            utilities[s] = 0
           
        #
        # END OF YOUR CODE
        #
        return policy, utilities
    
    
    # main method for value iteration - learn the policy for all the non final states
    
    def doLearning(self):
        
        # init the utilities vector
        auxUtilities = {}
        for s in self.fStates:
            auxUtilities[s] = self.problem.getReward(s)
        
        # Value Iteration main loop
        
        delta = math.inf
        it = 0
        print("It: ", end='')
        
        while (delta > self.convergenceThreshold): 
            it += 1
            print(str(it) + " ", end='')
            
            delta = 0
            exit = False;
            #
            # YOUR CODE HERE
            # Program the value iteration loop.
            # Hint: compute delta as the maximum difference between ut_i(state) and ut_{i+1}(state)
            #
            
            allStates = self.nfStates + self.fStates

            for s in allStates:

                if s in self.nfStates:
                    # Updating the utilities
                    maxVal = -math.inf

                    actions = self.problem.getPossibleActions(s)[0]
                    for a in actions:
                        distribution = self.problem.getTransitionDistribution(s, a)
                        value = 0
                        for d in distribution:
                            value += d[2] * self.utilities.get(d[1])
                    
                        if value > maxVal:
                            maxVal = value
                            self.policy[s] = a

                    # d[0] = action
                    # d[1] = resulting state
                    # d[2] = transition probability

                    auxUtilities[s] = self.problem.getReward(s) + self.gamma * maxVal

                    if abs(auxUtilities[s] - self.utilities[s]) > delta:
                        delta = abs(auxUtilities[s] - self.utilities[s])
                else:
                    auxUtilities[s] = self.problem.getReward(s)
                
            for s in self.nfStates:
                self.utilities[s] = auxUtilities[s]
            
            for s in self.fStates:
                self.utilities[s] = auxUtilities[s]

            #
            # END OF YOUR CODE
            #
                
        return self.policy


#### Class `QTable`

This class implements the QTable to be used by our QLearning algorithm.

We have implemented it as a dictionary, where each entry is a pair <state,q_values>. We will have an entry for each state "discovered/visited" by our agent and the corresponding utility values (q_values) learned for every posible action in that state. The q_values are also implemented by a dictionary of pairs <action,value>. Remember that in QLearning the environment is discovered during the learning phase, thus, only visited states are included in the QTable.

For your convenience, we provide all the methods needed to deal with this structure, so you don't have to modify this class, but if you feel more confortable by adding some additional method, please, feel free to do so. 

The provided methods are:

- `contains(state)` returns `True` if there exists an entry for the given state in the QTable. 

- `addEntry(state, possibleActions)` adds a new entry for the given `state` and its possible actions. The q-value is set to 0 for all the actions.

- `getValue(state, action)` returns the q-value for the pair <`state`,`action`>.

- `modifyEntry(state, action, value)` replaces the q-value of the <`state`,`action`> pair with the new value `value`.

- `getBestActon(state)` returns the action with maximum q-value for the given `state`.

- `getRandomAction(state)` returns a randomly selected action for the given `state`.

- `getOptimalPolicy()` returns a policy with the action of maximum q-value for each visited state (note that some states may not have been visited, so no action is provided for them).

In [25]:
# implemented as a dictionary with state as key and a dictionary with (action,q-value) as value

class QTable:
    
    def __init__(self):
        self.table = {}
        
    
    # check if the state is already included in the q-table
    
    def contains(self, state):
        return (state in self.table)
       
    
    # add entry
    
    def addEntry(self, state, posActions):
        qvalues = {}
        for act in posActions:
            qvalues[act.move] = 0.0
            
        self.table[state] = qvalues

    # get q-value
    
    def getValue(self, state, action):
        qvalues = self.table[state]
        return qvalues[action.move]
    
    # modify entry 
    
    def modifyEntry(self, state, action, value):
        qvalues = self.table[state]
        qvalues[action.move] = value
        
    # get best action for a given state
    
    def getBestAction(self, state):
        qvalues = self.table[state]
        bestQ = -math.inf
        bestAct = None
        for act, q in qvalues.items():
            if q > bestQ:
                bestQ = q
                bestAct = act
        
        return Action(bestAct)
    
    # get a random action for a given state
    
    def getRandomAction(self, state):
        qvalues = self.table[state]
        act, q = random.choice(list(qvalues.items()))
        
        return Action(act)
        
   
    # get policy from q-table
    
    def getOptimalPolicy(self):
        policy = {}
        
        for state, qvalues in self.table.items():
            action = self.getBestAction(state)
            policy[state] = action
        
        return policy
    
    # print the q-table
    
    def printQTable(self):
        for state, qvalues in self.table.items():
            print(f'{state}: ', end='')
            for action, value in qvalues.items():
                print(f'{action} {value} ', end='')
            print()
            

#### Class `QLearning`

This class will contain the Q-Learning method as explained in the lectures. You will have to code most part of this class. **Hint: follow as much as possible the pseudocode of the algorithm provided in the slides of unit 5.**

Remember that in this method we **cannot** exploit the knowledge about the environment (e.g. the states), the transition and reward models. We must discover the states during learning and ask the environment (`Problem` class) in order to know the result (state and reward) when applying an action in a given state.

Notice that a dictionary data structure is used to store the `policy` (entries are pairs <state,action>).


In [26]:
# Q-Learning: Reinforcement Learning by temporal difference

class QLearning:
    
    def __init__(self, problem, gamma, alpha, exploration, episodes):
        self.problem = problem
        self.gamma = gamma
        self.alpha = alpha
        self.exploration = exploration
        self.numEpisodes = episodes
        
        self.policy = {}
        
        self.qTable = QTable()
        self.initialState = self.problem.getInitialState()
        posActions, others = self.problem.getPossibleActions(self.initialState)
        self.qTable.addEntry(self.initialState, posActions)
        
    
    # main method for Q-learning - learn the policy
    
    def doLearning(self):

        for e in range(self.numEpisodes):
                 
            state = self.initialState
            newState = self.initialState

            while not self.problem.isFinal(state):
                
                #
                # YOUR CODE HERE
                # Implement Q-Learning
                #
                
                action = self.qTable.getBestAction(state)
                newState = self.problem.applyAction(state, action)
                reward = self.problem.getReward(newState)
                
                if not self.qTable.contains(newState):
                    self.qTable.addEntry(newState, self.problem.getPossibleActions(newState)[0])

                """
                maxNewVal = -math.inf
                for a in self.problem.getPossibleActions(newState)[0]:
                    if self.qTable.getValue(newState, a) > maxNewVal:
                        maxNewVal = self.qTable.getValue(newState, a)
                """

                maxNewVal = self.qTable.getValue(newState, self.qTable.getBestAction(newState))

                newValue = (1 - self.alpha) * self.qTable.getValue(state, action) + self.alpha * (reward + self.gamma * maxNewVal)

                self.qTable.modifyEntry(state, action, newValue)
                
                state = newState
                
                #
                # END OF YOUR CODE
                # 

        # self.qTable.printQTable()  
        # compute optimal policy
        
        self.policy = self.qTable.getOptimalPolicy()        
        return self.policy
        

The following method prints (in text) a given policy over the problem maze to be solved.

In [27]:
# prints the policy over the maze

def printPolicy(problem, policy):
    rows = problem.rows
    cols = problem.cols
    maze = problem.maze
    
    print()
    for row in range(rows):
        print()
        for col in range(cols):
            if ((maze[row][col] == 1)):
                print("■" + " ", end='')
            elif ((maze[row][col] == 3)):
                print("*" + " ", end='')
            else:
                state = State((row,col))
                if state in policy:
                    act = policy[state]
                    print(act.getChar() + " ", end='')
                else:
                    print("_ ", end='')
    print()
                
    

The following methods run a given policy a number of times and return the obtained (averaged) reward (utility). They are useful to evaluate the learnt policy. 


In [28]:
# runs an episode for the given policy. Returns the obtained utility

def singlePolicyRun(problem, policy, gamma):
   
    utility = 0.0
   
    state = problem.getInitialState()

    # 
    # YOUR CODE HERE
    #

    while not problem.isFinal(state):
        utility += gamma * problem.getReward(state)
        state = problem.applyAction(state, policy[state])
        
    utility += gamma * problem.getReward(state)
    # 
    # END OF YOU CODE
    #
    
    return utility


# runs a series of episodes for the given policy. Returns the averaged utility.

def multiplePolicyRun(problem, policy, gamma, numIterations):
    
    meanUtility = 0.0
    
    for it in range(numIterations):
        meanUtility += singlePolicyRun(problem, policy, gamma) 
                
    meanUtility /= numIterations
    
    return meanUtility 
    

#### The `main` function

Next, we provide you the `main` function that creates the problem and solves it using the policy learning algorithms. This method should be used afterwards to carry out the experimentation to study the behaviour of the implemented algorithms for different values of the parameters provided (size of the maze, maximum number of walls, number of gem cells, and algorithm).

In [29]:
def main(rows, cols, seed, maxDivisions, gemsCount, transitionProb, algorithm, gamma, convThreshold, alpha, exploration, numEpisodes):
    problem = Problem(rows, cols, seed, maxDivisions, gemsCount, transitionProb)

    print("$ python assignment3 " + str(problem.rows) + " " + str(problem.cols) + " " + str(seed) + " " 
          + str(maxDivisions) + " " + str(gemsCount) + " " +  str(transitionProb) + " " + 
          algorithm + " " + str(gamma) + " " + str(convThreshold) + " " + str(alpha) + " " +
          str(exploration) + " " + str(numEpisodes) + "\n")

    print("Problem instance:")
    printMaze(problem.maze)
    print("")
    
    # print gems values
    gems = problem.getGems()
    rewards = problem.getGemsRewards()
    
    for i in range(len(gems)):
        print("Gem at " + str(gems[i]) + " has a reward of: " + str(rewards[i]))

    learner = None

    if algorithm == "ValueIteration":
        learner = ValueIteration(problem,gamma,convThreshold)
    elif algorithm == "QLearning":
        learner = QLearning(problem, gamma, alpha, exploration, numEpisodes)
    else:
        raise Exception

    time_start = time.perf_counter()
    policy = learner.doLearning()
    time_end = time.perf_counter()
    print("")
    print("Elapsed time: " + str(time_end - time_start) + " seconds")

    return policy, problem

Below an example of scenario to be used

In [30]:
# Parameter List 

rows = 12
cols = 12
seed = 202
maxDivisions = 10
gemsCount = 5
transitionProb = 0.8 # deterministic, use e.g. 0.8 for a noisy environment

gamma = 0.9
threshold = 0.001
alpha = 0.8
exploration = 0 # no exploration is considered, the agent always behave greedily during learning
numEpisodes = 1000  # for learning
numIterations = 1000  # for evaluation

algorithm = "ValueIteration"
# algorithm = "QLearning"

policy, problem_instance = main(rows, cols, seed, maxDivisions, gemsCount, transitionProb, algorithm, gamma, threshold, alpha, exploration, numEpisodes)

printPolicy(problem_instance, policy)

ut = multiplePolicyRun(problem_instance, policy, gamma, numIterations)

print("\nObtained utility (over " + str(numIterations) + " iterations) for learnt policy is: " + str(ut))


$ python assignment3 12 12 202 10 5 0.8 ValueIteration 0.9 0.001 0.8 0 1000

Problem instance:
[0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]
[0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[1, 1, 3, 1, 1, 0, 1, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 1, 0, 2, 1, 0, 0]
[0, 0, 0, 1, 0, 3, 1, 0, 1, 1, 0, 1]
[1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0]
[0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0]
[3, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
[0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]
[0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]

Gem at Position: (3, 2) has a reward of: 40
Gem at Position: (4, 4) has a reward of: 0
Gem at Position: (6, 5) has a reward of: 0
Gem at Position: (8, 6) has a reward of: 25
Gem at Position: (9, 0) has a reward of: 0
It: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 
Elapsed time: 0.14504009998927359 seconds


↓ ↓ ↓ ■ ↓ ↓ ■ ↓ ↓ ■ ↓ ↓ 
→ → ↓ ■ ↓ ← ■ ↓ ↓ ■ ↓ ↓ 
→ → ↓ ← ← ← ← ← ← ■ ↓ ↓ 
■ ■ * ■ ■ ↑ ■ 

#### Printing the result

Here we provide you some code to display the maze and the path carried out by the robot (the green cell) to solve the instance of the problem. Walls are represented as black cells, and garbage with brown cells.

In [31]:
def render_maze(grid, maze, gems, gemsRewards):
    height, width = len(maze), len(maze[0])
    
    # Render maze
    for i in range(width):
        for j in range(height):
            grid[j,i] = colors.LightGray if maze[j][i] in [0,2,3] else colors.Black

            
    # Render gems in maze --> brightness based on reward    
    purpleMaxDif = [a - b for a, b in zip(colors.Violet, colors.Purple)]    
    maxReward = max(gemsRewards)
    minReward = min(gemsRewards)
    RewardRange = maxReward - minReward
    for gem, reward in zip(gems, gemsRewards):
        weight = float(reward - minReward) / float(RewardRange)
        grid[gem.pos] = [int(a + b*weight) for a,b in zip(colors.Purple, purpleMaxDif)]
        
def find_agent(maze, width, height):
    for i in range(width):
        for j in range(height):
            if (maze[j][i] == 2):
                return (i,j)

def render_problem(policy, problem_instance, gamma = 1.0):
    height, width = len(problem_instance.maze), len(problem_instance.maze[0])
    solution_grid = BlockGrid(width, height, fill=colors.LightGray, lines_on=True)
    agentPos = find_agent(problem_instance.maze, width, height)
    
    # Initial position rendering
    render_maze(solution_grid, problem_instance.maze, problem_instance.gems, problem_instance.gemsRewards)
    solution_grid[agentPos[1],agentPos[0]] = colors.Green
    solution_grid.show()
    
    agent_state = State((agentPos[1], agentPos[0]))
    
    g = gamma
    numSteps = 0
    reward = 0
        
    while (agent_state not in problem_instance.gems):
        agent_state = problem_instance.applyAction(agent_state, policy[agent_state])
        
        reward += g*problem_instance.getReward(agent_state)
        g *= g
        numSteps += 1
        
        # Render maze
        render_maze(solution_grid, problem_instance.maze, problem_instance.gems, problem_instance.gemsRewards)
        
        # Render agent, and update its position and the garbage list
        solution_grid[agent_state.pos[0], agent_state.pos[1]] = colors.Green
        
        solution_grid.show()
        clear_output(wait=True)
        time.sleep(0.2)
        
    print(f'Obtained utility after {numSteps} steps and reaching gem with reward {problem_instance.getReward(agent_state)}: {reward}')
    solution_grid.show()

In [32]:
# render_problem(policy, problem_instance)

## Experimental results

Once the algorithms have been implemented, you must study their performance. In order to do that, you must compare the quality of the solutions obtained, as well as the number of expanded nodes for instances of different maze sizes, number of walls and number of garbage cells.

Please, use new cells to insert code to carry out the experimental results and study of the algorithms.